<a href="https://colab.research.google.com/github/laurenneal/capstone-visual-neuroscience/blob/main/basic_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import sys
from more_itertools import sliced
import os

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import tensorflow_probability as tfp
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

from google.colab import drive

# references
# https://colab.research.google.com/github/keras-team/keras-io/blob/master/examples/keras_recipes/ipynb/bayesian_neural_networks.ipynb#scrollTo=nB9dOAJYJY79
# https://www.tensorflow.org/tutorials/load_data/pandas_dataframe # from pandas to TF dataset
# https://machinelearningmastery.com/multi-class-classification-tutorial-keras-deep-learning-library/ # set up (0ne hot encoding label data)

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!ls drive/MyDrive/DS6011_Capstone_VisualNeuroscience/Seeded_CNMF/Extracted_Features/5_frames_stim1

HOLDOUT_RAW_extracted_data_features.csv
TrainTest_RAW_extracted_data_features.csv


In [4]:
# pull all data from folder
root = '/content/drive/MyDrive/DS6011_Capstone_VisualNeuroscience/Seeded_CNMF/Extracted_Features/'
folder = '5_frames_stim1/'
f_names = os.listdir(root + folder)

for f in f_names:
  if 'HOLDOUT' in f:
    df_HOLD = pd.read_csv(root + folder + f)
  elif "TrainTest" in f:
    df = pd.read_csv(root + folder + f)

In [5]:
df

,Unnamed: 0,stack,label,roi,frame,movie_ID,resp,stim1,stim2,stim3,...,relation_stim1_PS-PS,relation_type_stim6,relation_stim6,relation_stim6_+5,relation_type_stim6_+5,relation_stim6_NS-NS,relation_stim6_NS-PS,relation_stim6_PS-NS,relation_stim6_PS-PS,frame_group
0,0,1,tm9,1,0,210728_0,0.119422,1.827970,-0.668979,-1.323903,...,0.0,PS,6.131515e-15,1.709783e-19,NS,0.000000e+00,0.0,1.048356e-33,0.000000e+00,1
1,1,1,tm9,1,1,210728_0,0.065380,0.676591,1.133205,0.209498,...,0.0,PS,1.404623e-10,2.707063e-14,NS,0.000000e+00,0.0,3.802402e-24,0.000000e+00,1
2,2,1,tm9,1,2,210728_0,0.036917,-1.276472,-0.881429,1.220991,...,0.0,PS,7.426788e-10,2.085457e-13,NS,0.000000e+00,0.0,1.548825e-22,0.000000e+00,1
3,3,1,tm9,1,3,210728_0,0.156975,-1.676237,0.505447,1.208871,...,0.0,PS,1.971844e-08,9.885126e-13,NS,0.000000e+00,0.0,1.949192e-20,0.000000e+00,1
4,4,1,tm9,1,4,210728_0,0.195883,-1.351235,-0.832430,1.249990,...,0.0,PS,3.232196e-08,1.051521e-12,NS,0.000000e+00,0.0,3.398724e-20,0.000000e+00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3688505,3688505,9,tm1,10,5490,210816_1,0.074290,0.570051,0.754433,0.311900,...,0.0,PS,3.263303e-09,2.459124e-13,PS,0.000000e+00,0.0,0.000000e+00,8.024868e-22,10
3688506,3688506,9,tm1,10,5491,210816_1,0.086649,0.570051,0.772721,0.007225,...,0.0,PS,3.979294e-09,2.972849e-13,PS,0.000000e+00,0.0,0.000000e+00,1.182984e-21,10
3688507,3688507,9,tm1,10,5492,210816_1,0.034598,0.570051,0.772721,0.007225,...,0.0,PS,6.265856e-11,2.466365e-17,NS,0.000000e+00,0.0,1.545389e-27,0.000000e+00,10
3688508,3688508,9,tm1,10,5497,210816_1,0.229270,-1.064034,-1.657355,1.230574,...,0.0,NS,1.621214e-09,2.750217e-14,NS,4.458690e-23,0.0,0.000000e+00,0.000000e+00,10


In [6]:
df_HOLD

,Unnamed: 0,stack,label,roi,frame,movie_ID,resp,stim1,stim2,stim3,...,relation_stim1_PS-PS,relation_type_stim6,relation_stim6,relation_stim6_+5,relation_type_stim6_+5,relation_stim6_NS-NS,relation_stim6_NS-PS,relation_stim6_PS-NS,relation_stim6_PS-PS,frame_group
0,0,1,t5,8,1,210809_2,0.217502,0.276790,0.776442,-0.001301,...,0.000000e+00,PS,7.216378e-11,2.790717e-16,NS,0.0,0.0,2.013887e-26,0.000000e+00,1
1,1,1,t5,8,2,210809_2,0.416828,0.276790,0.776442,-0.001301,...,0.000000e+00,PS,1.653759e-08,9.894077e-13,NS,0.0,0.0,1.636242e-20,0.000000e+00,1
2,2,1,t5,8,3,210809_2,0.354376,-0.551717,-1.050625,-1.773261,...,0.000000e+00,PS,1.354199e-07,2.692603e-11,NS,0.0,0.0,3.646319e-18,0.000000e+00,1
3,3,1,t5,8,4,210809_2,0.502063,-1.104055,-1.343779,-1.581956,...,0.000000e+00,PS,8.086540e-07,2.216457e-10,NS,0.0,0.0,1.792346e-16,0.000000e+00,1
4,4,1,t5,8,5,210809_2,0.332470,-1.104055,0.776442,-0.001301,...,0.000000e+00,PS,5.285532e-09,6.543285e-14,NS,0.0,0.0,3.458474e-22,0.000000e+00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
691492,691492,10,tm9,1,5497,210816_0,0.012834,0.723927,-1.568715,-1.344984,...,0.000000e+00,PS,2.906709e-09,8.282521e-14,PS,0.0,0.0,0.000000e+00,2.407488e-22,10
691493,691493,10,tm9,1,5498,210816_0,0.017510,0.660119,0.410886,-1.260188,...,0.000000e+00,PS,1.235647e-08,6.317373e-13,PS,0.0,0.0,0.000000e+00,7.806041e-21,10
691494,691494,10,tm9,1,5499,210816_0,0.071367,0.660119,0.772401,0.007249,...,0.000000e+00,PS,1.903339e-07,8.094906e-12,PS,0.0,0.0,0.000000e+00,1.540735e-18,10
691495,691495,10,tm9,1,5500,210816_0,0.046596,0.660119,0.772401,0.007249,...,2.757963e-17,PS,6.542141e-08,1.857758e-12,PS,0.0,0.0,0.000000e+00,1.215371e-19,10


In [7]:
# Save labels as seperate obj
Y = df.pop('label') # label data (removes label from df)
Y_HOLD = df_HOLD.pop('label') # label data (removes label from df)

In [8]:

## SET UP LABEL DATA

# format for TF

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)


## SET UP LABEL DATA FOR HOLDOUT

# format for TF

# encode class values as integers
encoder2 = LabelEncoder()
encoder2.fit(Y_HOLD)
encoded_Y_HOLD = encoder.transform(Y_HOLD)
# convert integers to dummy_HOLD variables (i.e. one hot encoded)
dummy_y_HOLD = np_utils.to_categorical(encoded_Y_HOLD)

In [9]:
# NOTE: here we copy only the numeric cells. This is a short term fix to get it working.
FEATURE_NAMES = [
       'stim2_feature', 'stim3_feature', 'stim5_feature', 'stim4_feature',
       'stim7_feature', 'stim8_feature', 
       'relation_stim1', 'relation_stim1_+5',
       'relation_stim1_NS-NS', 'relation_stim1_NS-PS', 'relation_stim1_PS-NS',
       'relation_stim1_PS-PS',  'relation_stim6',
       'relation_stim6_+5',  'relation_stim6_NS-NS',
       'relation_stim6_NS-PS', 'relation_stim6_PS-NS', 'relation_stim6_PS-PS',
] # too many features? Are these the right ones?????
# !!!!!!!!!! need to deal with strings here

######### remove feature names from this cell when we've figured out how to include index info

# format for TF

dataset = tf.data.Dataset.from_tensor_slices((dict(df[FEATURE_NAMES]), dummy_y)) # converted

# same for holdout data

dataset_HOLD = tf.data.Dataset.from_tensor_slices((dict(df_HOLD[FEATURE_NAMES]), dummy_y_HOLD)) # converted

In [10]:
def get_train_and_test_splits(dataset, train_size, batch_size=1):
    # We prefetch with a buffer the same size as the dataset because th dataset

    # We shuffle with a buffer
    train_dataset = (
        dataset.take(train_size).shuffle(buffer_size=train_size).batch(batch_size)
    )
    test_dataset = dataset.skip(train_size).batch(batch_size)

    return train_dataset, test_dataset

In [11]:
hidden_units = [8, 8]
learning_rate = 0.001


def run_experiment(model, loss, train_dataset, test_dataset):

    model.compile(
        optimizer=keras.optimizers.RMSprop(learning_rate=learning_rate),
        loss=loss,
        metrics=[keras.metrics.RootMeanSquaredError()],
    )

    print("Start training the model...")
    model.fit(train_dataset, epochs=num_epochs, validation_data=test_dataset)
    print("Model training finished.")
    _, rmse = model.evaluate(train_dataset, verbose=0)
    print(f"Train RMSE: {round(rmse, 3)}")

    print("Evaluating model performance...")
    _, rmse = model.evaluate(test_dataset, verbose=0)
    print(f"Test RMSE: {round(rmse, 3)}")


In [12]:
df.columns

Index(['Unnamed: 0', 'stack', 'roi', 'frame', 'movie_ID', 'resp', 'stim1',
       'stim2', 'stim3', 'stim4', 'stim5', 'stim6', 'stim7', 'stim8',
       'stim2_feature', 'stim3_feature', 'stim5_feature', 'stim4_feature',
       'stim7_feature', 'stim8_feature', 'relation_type_stim1',
       'relation_stim1', 'relation_stim1_+5', 'relation_type_stim1_+5',
       'relation_stim1_NS-NS', 'relation_stim1_NS-PS', 'relation_stim1_PS-NS',
       'relation_stim1_PS-PS', 'relation_type_stim6', 'relation_stim6',
       'relation_stim6_+5', 'relation_type_stim6_+5', 'relation_stim6_NS-NS',
       'relation_stim6_NS-PS', 'relation_stim6_PS-NS', 'relation_stim6_PS-PS',
       'frame_group'],
      dtype='object')

In [13]:
df.dtypes

Unnamed: 0                  int64
stack                       int64
roi                         int64
frame                       int64
movie_ID                   object
resp                      float64
stim1                     float64
stim2                     float64
stim3                     float64
stim4                     float64
stim5                     float64
stim6                     float64
stim7                     float64
stim8                     float64
stim2_feature             float64
stim3_feature             float64
stim5_feature             float64
stim4_feature             float64
stim7_feature             float64
stim8_feature             float64
relation_type_stim1        object
relation_stim1            float64
relation_stim1_+5         float64
relation_type_stim1_+5     object
relation_stim1_NS-NS      float64
relation_stim1_NS-PS      float64
relation_stim1_PS-NS      float64
relation_stim1_PS-PS      float64
relation_type_stim6        object
relation_stim6

In [14]:
# FEATURE_NAMES = [
#        'stim2_feature', 'stim3_feature', 'stim5_feature', 'stim4_feature',
#        'stim7_feature', 'stim8_feature', 
#        'relation_stim1', 'relation_stim1_+5',
#        'relation_stim1_NS-NS', 'relation_stim1_NS-PS', 'relation_stim1_PS-NS',
#        'relation_stim1_PS-PS',  'relation_stim6',
#        'relation_stim6_+5',  'relation_stim6_NS-NS',
#        'relation_stim6_NS-PS', 'relation_stim6_PS-NS', 'relation_stim6_PS-PS',
# ] # too many features? Are these the right ones?????
# # !!!!!!!!!! need to deal with strings here


def create_model_inputs():
    inputs = {}
    for feature_name in FEATURE_NAMES:
        inputs[feature_name] = layers.Input(
            name=feature_name, shape=(1,), dtype=tf.float32
        )
    return inputs


In [18]:
def create_baseline_model():
    inputs = create_model_inputs()
    print(inputs)
    input_values = [value for _, value in sorted(inputs.items())]
    features = keras.layers.concatenate(input_values)
    features = layers.BatchNormalization()(features)

    # Create hidden layers with deterministic weights using the Dense layer.
    for units in hidden_units:
        features = layers.Dense(units, activation="sigmoid")(features)
    # The output is deterministic: a single point estimate.
    outputs = layers.Dense(units=5)(features)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


In [16]:
dataset_size = 691497
batch_size = 256
train_size = int(dataset_size * 0.85)
train_dataset, test_dataset = get_train_and_test_splits(dataset,train_size,batch_size)

In [ ]:
num_epochs = 100
cat_crossent = keras.losses.CategoricalCrossentropy()
baseline_model = create_baseline_model()
run_experiment(baseline_model, cat_crossent, train_dataset, test_dataset) # DON'T USE MSE

{'stim2_feature': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'stim2_feature')>, 'stim3_feature': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'stim3_feature')>, 'stim5_feature': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'stim5_feature')>, 'stim4_feature': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'stim4_feature')>, 'stim7_feature': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'stim7_feature')>, 'stim8_feature': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'stim8_feature')>, 'relation_stim1': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'relation_stim1')>, 'relation_stim1_+5': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'relation_stim1_+5')>, 'relation_stim1_NS-NS': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'relation_stim1_NS-NS')>, 'relation_stim1_NS-PS': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'relati

In [ ]:
sample = 10
examples, targets = list(test_dataset.unbatch().shuffle(batch_size * 10).batch(sample))[
    0
]

predicted = baseline_model(examples).numpy()
for idx in range(sample):
    print(f"Predicted: {round(float(predicted[idx][0]), 1)} - Actual: {targets[idx]}")

In [12]:
# 1. average results by roi
# 2. Test on holdout data (avg out by roi again)
# 3. Try the bayesian version

<TensorSliceDataset element_spec=({'stim2_feature': TensorSpec(shape=(), dtype=tf.float64, name=None), 'stim3_feature': TensorSpec(shape=(), dtype=tf.float64, name=None), 'stim5_feature': TensorSpec(shape=(), dtype=tf.float64, name=None), 'stim4_feature': TensorSpec(shape=(), dtype=tf.float64, name=None), 'stim7_feature': TensorSpec(shape=(), dtype=tf.float64, name=None), 'stim8_feature': TensorSpec(shape=(), dtype=tf.float64, name=None), 'relation_stim1': TensorSpec(shape=(), dtype=tf.float64, name=None), 'relation_stim1_+5': TensorSpec(shape=(), dtype=tf.float64, name=None), 'relation_stim1_NS-NS': TensorSpec(shape=(), dtype=tf.float64, name=None), 'relation_stim1_NS-PS': TensorSpec(shape=(), dtype=tf.float64, name=None), 'relation_stim1_PS-NS': TensorSpec(shape=(), dtype=tf.float64, name=None), 'relation_stim1_PS-PS': TensorSpec(shape=(), dtype=tf.float64, name=None), 'relation_stim6': TensorSpec(shape=(), dtype=tf.float64, name=None), 'relation_stim6_+5': TensorSpec(shape=(), dtype